In [ ]:
#
import json
from airflow.operators.bash import BashOperator
from airflow.operators.dummy import DummyOperator
from airflow.operators.python import PythonOperator
from airflow.utils.dates import days_ago
from airflow import DAG

from airflow.providers.http.operators.http import SimpleHttpOperator

def show_status(result1,result2):
    print('All result from rest api compute')
    print('result1, = ', result1,)
    print('result2, = ', result2,)
    print('--- DONE ---')

with DAG('rest_fastapi', start_date=days_ago(1)) as dag:

    post_op1 = SimpleHttpOperator(
        task_id='post_op1',
        endpoint='compute',
        http_conn_id='localhttp',
        data=json.dumps({"m": 2, 'x': 14, 'c': 3}),
        headers={"Content-Type": "application/json"},
        response_check=lambda response: response.json()['result'] == 31
    )
    post_op2 = SimpleHttpOperator(
    task_id='post_op2',
    endpoint='compute',
    http_conn_id='localhttp',
    data=json.dumps({"m": -0.5, 'x': 3, 'c': 0}),
    headers={"Content-Type": "application/json"},
    response_check=lambda response: response.json()['result'] == -1.5
)
    # task_get_op = SimpleHttpOperator(
    # task_id="get_op",
    # method="GET",
    # endpoint="/",
    # data={"param1": "value1"},
    # headers={},
    #     )
    inform = PythonOperator(task_id='inform_status1', python_callable=show_status, op_args=[post_op1.output,post_op2.output])
    start = DummyOperator(task_id='start_task')
  
    start >> [post_op1,post_op2] >>inform
    

In [ ]:
from typing import Union
from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn

app = FastAPI()

class Item(BaseModel):
    x: float
    m: float
    c: float

@app.post("/compute")
async def login(item:Item):
    y = (item.m*item.x)+item.c
    return{"result":y}

@app.get("/")
async def read_root():
    return {"Hello": "World"}

if __name__ == "__main__":
    uvicorn.run('fastapi-compute:app', host="0.0.0.0", port=8000, reload=True)